# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citi_df = pd.read_csv("CitiWeather_Output.csv")
citi_df.head()

,Unnamed: 0,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dingle,48.247690,-14.052342,82.33,77.0,100.0,3.40,PH,1.601428e+09
1,1,busselton,-61.263477,87.635957,64.00,80.0,99.0,10.42,AU,1.601428e+09
2,2,skala fourkas,39.701840,23.407450,62.01,59.0,0.0,8.99,GR,1.601428e+09
3,3,bonavista,51.274296,-44.646373,60.44,92.0,46.0,17.76,CA,1.601428e+09
4,4,albany,-57.070868,104.717368,61.00,84.0,100.0,3.44,US,1.601428e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations, store humidities in weights
locations = citi_df[["Latitude", "Longitude"]]
weights = citi_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
citi_cleaned = citi_df[["Cities","Latitude","Longitude","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
citi_cleaned["Lat"] = ""
citi_cleaned["Lng"] = ""
citi_cleaned.head()

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng
0,dingle,48.247690,-14.052342,82.33,77.0,100.0,3.40,PH,1.601428e+09,,
1,busselton,-61.263477,87.635957,64.00,80.0,99.0,10.42,AU,1.601428e+09,,
2,skala fourkas,39.701840,23.407450,62.01,59.0,0.0,8.99,GR,1.601428e+09,,
3,bonavista,51.274296,-44.646373,60.44,92.0,46.0,17.76,CA,1.601428e+09,,
4,albany,-57.070868,104.717368,61.00,84.0,100.0,3.44,US,1.601428e+09,,


In [6]:
#Retrieve the cities Lat and Lng with geocoding api
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
params = {
    "key":g_key
}
for index, row in citi_cleaned.iterrows():
    city = row["Cities"]
    country = row["Country"]
    params["address"] = f"{city},{country}"
    data = requests.get(base_url,params=params).json()
  
    try: 
        results = data["results"][0]["geometry"]["location"]
        citi_cleaned.loc[index,"Lat"] = results["lat"]
        citi_cleaned.loc[index,"Lng"] = results["lng"]
    except (KeyError, IndexError):
        pass


In [7]:
citi_cleaned.head()

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng
0,dingle,48.247690,-14.052342,82.33,77.0,100.0,3.40,PH,1.601428e+09,11.0111,122.65
1,busselton,-61.263477,87.635957,64.00,80.0,99.0,10.42,AU,1.601428e+09,-33.6532,115.345
2,skala fourkas,39.701840,23.407450,62.01,59.0,0.0,8.99,GR,1.601428e+09,40.0057,23.4149
3,bonavista,51.274296,-44.646373,60.44,92.0,46.0,17.76,CA,1.601428e+09,48.6533,-53.1118
4,albany,-57.070868,104.717368,61.00,84.0,100.0,3.44,US,1.601428e+09,42.6526,-73.7562


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
citi_cleaned["Hotel Name"]=""
hotel_df = citi_cleaned
hotel_df.head()

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng,Hotel Name
0,dingle,48.247690,-14.052342,82.33,77.0,100.0,3.40,PH,1.601428e+09,11.0111,122.65,
1,busselton,-61.263477,87.635957,64.00,80.0,99.0,10.42,AU,1.601428e+09,-33.6532,115.345,
2,skala fourkas,39.701840,23.407450,62.01,59.0,0.0,8.99,GR,1.601428e+09,40.0057,23.4149,
3,bonavista,51.274296,-44.646373,60.44,92.0,46.0,17.76,CA,1.601428e+09,48.6533,-53.1118,
4,albany,-57.070868,104.717368,61.00,84.0,100.0,3.44,US,1.601428e+09,42.6526,-73.7562,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "key":g_key,
    "radius":50000,
    "keyword":"hotel"
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    data = requests.get(base_url, params=params).json()
    results = data["results"]
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [10]:
# Save Data to csv
hotel_df.to_csv("Hotel.csv")

# Visualize to confirm airport data appears
hotel_df.head(10)

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng,Hotel Name
0,dingle,48.247690,-14.052342,82.33,77.0,100.0,3.40,PH,1.601428e+09,11.0111,122.65,Urban Sands Iloilo
1,busselton,-61.263477,87.635957,64.00,80.0,99.0,10.42,AU,1.601428e+09,-33.6532,115.345,Pullman Bunker Bay Resort Margaret River Region
2,skala fourkas,39.701840,23.407450,62.01,59.0,0.0,8.99,GR,1.601428e+09,40.0057,23.4149,Art Boutique Hotel
3,bonavista,51.274296,-44.646373,60.44,92.0,46.0,17.76,CA,1.601428e+09,48.6533,-53.1118,Seaport Inn Hotel & Restaurant
4,albany,-57.070868,104.717368,61.00,84.0,100.0,3.44,US,1.601428e+09,42.6526,-73.7562,The Maker Hotel
5,taolanaro,-41.173635,56.817313,NaN,NaN,NaN,NaN,NaN,NaN,-25.0225,46.9854,Manafiafy Beach & Rainforest Lodge
6,clyde river,69.956380,-70.692016,28.40,79.0,90.0,11.41,CA,1.601428e+09,70.4764,-68.6013,Naujaaraaluit – Clyde River Hotel
7,tautira,-17.085562,-141.871165,87.80,55.0,40.0,6.93,PF,1.601428e+09,-17.7496,-149.161,InterContinental Resort Tahiti
8,carnarvon,-25.945395,109.640780,66.20,59.0,0.0,16.11,AU,1.601428e+09,-24.8877,113.662,Coral Coast Tourist Park
9,mehamn,79.072578,27.074957,49.01,84.0,31.0,10.36,NO,1.601428e+09,71.0357,27.8489,Mehamn Arctic Hotel AS


In [4]:
hotel_df = pd.read_csv("Hotel.csv")
hotel_df.head(100)

,Unnamed: 0,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat,Lng,Hotel Name
0,0,dingle,48.247690,-14.052342,82.33,77.0,100.0,3.40,PH,1.601428e+09,11.011131,122.650351,Urban Sands Iloilo
1,1,busselton,-61.263477,87.635957,64.00,80.0,99.0,10.42,AU,1.601428e+09,-33.653158,115.345489,Pullman Bunker Bay Resort Margaret River Region
2,2,skala fourkas,39.701840,23.407450,62.01,59.0,0.0,8.99,GR,1.601428e+09,40.005744,23.414934,Art Boutique Hotel
3,3,bonavista,51.274296,-44.646373,60.44,92.0,46.0,17.76,CA,1.601428e+09,48.653345,-53.111825,Seaport Inn Hotel & Restaurant
4,4,albany,-57.070868,104.717368,61.00,84.0,100.0,3.44,US,1.601428e+09,42.652579,-73.756232,The Maker Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,lasa,29.023230,90.404080,45.00,94.0,65.0,4.79,IT,1.601428e+09,NaN,NaN,NaN
96,96,severo-kurilsk,35.934230,166.655170,55.15,66.0,100.0,9.73,RU,1.601428e+09,50.668342,156.115192,Kak Doma
97,97,ushuaia,-67.886462,-39.259028,33.80,80.0,40.0,13.87,AR,1.601428e+09,-54.801912,-68.302951,Australis Kauyeken
98,98,ushuaia,-81.279776,-31.757114,33.80,80.0,40.0,13.87,AR,1.601428e+09,-54.801912,-68.302951,Australis Kauyeken


In [27]:
hotel_df.count()

Unnamed: 0    500
Cities        500
Latitude      500
Longitude     500
Max Temp      464
Humidity      464
Cloudiness    464
Wind Speed    464
Country       463
Date          464
Lat           485
Lng           485
Hotel Name    463
dtype: int64

In [5]:
hotel_df = hotel_df.dropna(how="any")
hotel_df.count()

Unnamed: 0    435
Cities        435
Latitude      435
Longitude     435
Max Temp      435
Humidity      435
Cloudiness    435
Wind Speed    435
Country       435
Date          435
Lat           435
Lng           435
Hotel Name    435
dtype: int64

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
locations.astype(float)

,Lat,Lng
0,11.011131,122.650351
1,-33.653158,115.345489
2,40.005744,23.414934
3,48.653345,-53.111825
4,42.652579,-73.756232
...,...,...
495,-20.234957,164.032950
496,18.429839,-65.877450
497,-45.571225,-72.068265
498,42.652579,-73.756232


In [8]:
humidity = hotel_df["Humidity"].astype(float)

In [10]:
import gmaps.datasets
# Add marker layer ontop of heat map
# Plot Heatmap

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Add layer

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))